In [ ]:
import pandas as pd
from common.base import connection_oracle
from sqlalchemy import (MetaData,Table,String,
Integer,Float,Column,Numeric,inspect,select,insert)

#SqlAlchemy
engine = connection_oracle()
metadata = MetaData()
connection = engine.connect()
insp = inspect(engine)

In [ ]:
def table_ibge():

    municipios_ibge = pd.read_excel('./src/database/db_ibge.xls',
    usecols=['Código Município Completo','UF','Nome_Município'])

    ibge_estados = pd.read_excel('./src/database/db_ibge_estado.xlsx')

    #Mudando Nome da Coluna do IBGE e Estados
    municipios_ibge\
        .rename(columns={'Código Município Completo':'mun',
        'Nome_Município':'municipio','UF':'cod_uf'},inplace=True)

    ibge_estados\
        .rename(columns={'Código da UF':'cod_uf','Estado':"estado"},inplace=True)


    #Deixando Municipios e Estado em extenso em letra maiscula
    municipios_ibge.municipio = municipios_ibge\
        .municipio.str.upper()

    ibge_estados.estado = ibge_estados\
        .estado.str.upper()

    #Cruzando com IBGE com siglas de Municipio
    ibge = municipios_ibge.merge(ibge_estados,on='cod_uf')

    #Removendo linhas duplicadas do datafrmae
    ibge.drop_duplicates(inplace=True)

    return ibge
def table_bdgd():
    table = Table('db_raw_bdgd_ucmt',metadata,autoload_with=engine)
    stmt = select(table)
    results= connection.execute(stmt).fetchall()
    dataframe = pd.DataFrame(results)
    dataframe.columns = [col.get('name') for col in insp.get_columns('db_raw_bdgd_ucmt')]
    return dataframe
def create_table(name):
    bdgd_at = Table(f'{name}',metadata,
             Column('COD_ID', String(255)),
             Column('DIST', String(255)),
             Column('LGRD', String(255)),
             Column('BRR', String(255)),
             Column('MUNICIPIO', String(255)),
             Column('ESTADO', String(255)),
             Column('UF', String(255)),
             Column('COD_UF', String(10)),
             Column('MUN', Numeric),
             Column('CEP', String(255)),
             Column('CNAE', String(255)),
             Column('ENE_MED', Float),
             Column('DEM_MAX', Float),
             Column('DEM_CONT',Float),
             Column('LIV', String(5)),
             Column('GRU_TAR',String(255)),
             Column('SIT_ATIV', String(255)),
             Column('CLAS_SUB', String(255)),
             Column('DAT_CON', String(255)))

    metadata.create_all(engine)
    return bdgd_at

df_ibge = table_ibge()
df_bdgd = table_bdgd()

In [ ]:
print(f"bdgd{df_bdgd.shape} | ibge {df_ibge.shape}")

In [ ]:
#Relationship codigo_mun bdgd com df_ibge
dataframe_raw = df_bdgd.merge(df_ibge,on='mun',
how='left',copy=True)
print(f"Cruzamento{dataframe_raw.shape} ")

In [ ]:
#Upper Columns
dataframe_raw.columns  = dataframe_raw.keys().str.upper()

In [ ]:
dataframe_raw

In [ ]:
#Calculando - Consumo Médio 12 Meses
dataframe_raw['ENE_MED'] = dataframe_raw.loc[:, 'ENE_01':'ENE_12'].mean(axis=1)
#Calculando - Demanda Maxima 12 Meses
dataframe_raw['DEM_MAX'] = dataframe_raw.loc[:, 'DEM_01':'DEM_12'].max(axis=1)
#Calculando - Total Horas Sem Energia nos 12 Meses
dataframe_raw['DIC'] = dataframe_raw.loc [:,'DIC_01':'DIC_12'].sum(axis=1)
#Calculando - Total de Interrupções nos 12 Meses
dataframe_raw['FIC'] = dataframe_raw.loc [:,'FIC_01':'FIC_12'].sum(axis=1)

In [ ]:
#Excluindo Colunas
dataframe_raw.drop(['DEM_01', 'DEM_02', 'DEM_03', 'DEM_04', 'DEM_05', 'DEM_06', 'DEM_07',
                     'DEM_08', 'DEM_09', 'DEM_10','DEM_11', 'DEM_12','ENE_01', 'ENE_02',
                     'ENE_03', 'ENE_04', 'ENE_05', 'ENE_06', 'ENE_07', 'ENE_08', 'ENE_09',
                     'ENE_10', 'ENE_11', 'ENE_12','DIC_01', 'DIC_02', 'DIC_03', 'DIC_04',
                     'DIC_05','DIC_06', 'DIC_07', 'DIC_08', 'DIC_09', 'DIC_10', 'DIC_11',
                     'DIC_12','FIC_01', 'FIC_02', 'FIC_03', 'FIC_04', 'FIC_05', 'FIC_06',
                     'FIC_07','FIC_08', 'FIC_09', 'FIC_10', 'FIC_11', 'FIC_12','DESCR'],axis= 1 ,inplace=True)

In [ ]:
#Rename CTAT_CTMT
dataframe_raw.rename(columns={'CTAT':'CT_AT_MT'},inplace=True)

In [ ]:
 #Create Table SQL
bdgd_mt = create_table('bdgd_mt_test')

In [ ]:
# Loc columns to commit 
df_bdgd_mt = dataframe_raw\
    .loc[:,['COD_ID','DIST','LGRD','BRR','MUNICIPIO','ESTADO',
            'UF','COD_UF','MUN','CEP','CNAE','ENE_MED',
            'DEM_MAX','DEM_CONT','LIV','GRU_TAR','SIT_ATIV',
            'CLAS_SUB','DAT_CON']].copy()

In [ ]:
#Tranform List-Dict to SqlAlchemy
values_list =[{'COD_ID':row.COD_ID,'DIST':row.DIST,'LGRD':row.LGRD, 'BRR':row.BRR,
  'MUNICIPIO':row.MUNICIPIO, 'ESTADO':row.ESTADO, 'UF':row.UF,
  'COD_UF':row.COD_UF, 'MUN':row.MUN, 'CEP':row.CEP,
  'CNAE':row.CNAE, 'ENE_MED':row.ENE_MED, 'DEM_MAX':row.DEM_MAX,
  'LIV':row.LIV, 'GRU_TAR':row.GRU_TAR,'SIT_ATIV':row.SIT_ATIV,
  'CLAS_SUB':row.CLAS_SUB,'DAT_CON':row.DAT_CON} for row in df_bdgd_mt.itertuples()]

In [26]:
#Commit Oracle - SQL 
stmt = insert(bdgd_mt)
results_proxy = connection.execute(stmt,values_list)
connection.commit()

print(results_proxy.rowcount)

7344
